In [1]:
# default_exp validation

In [2]:
# hide
%reload_ext lab_black

# Geometry Validation

> geometry validations

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/thinkingmachines/geowrangler/blob/master/notebooks/00_validation.ipynb)


In [3]:
# exporti
import logging
import warnings
from abc import ABC, abstractmethod
from typing import Sequence, Union

import geopandas as gpd
import pandas as pd
from fastcore.basics import patch
from shapely import validation as shapely_validation
from shapely.algorithms.cga import signed_area
from shapely.geometry.base import BaseGeometry
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry.polygon import orient

logger = logging.getLogger(__name__)

In [4]:
# export
class ValidationError(Exception):
    pass

In [5]:
# export
class BaseValidator(ABC):
    """Abstract Base Class for single validator"""

    fix_available = True
    warning_message = "Geometry errors found"
    geometry_types = None

    def __init__(
        self,
        add_new_column: bool = True,
        apply_fix: bool = True,
    ):
        self.add_new_column = add_new_column
        self.apply_fix = apply_fix

    @property
    @abstractmethod
    def validator_column_name(self):  # pragma: no cover
        pass

    def get_check_arguments(self, gdf: gpd.GeoDataFrame) -> dict:
        return {}

    def check(
        self, geometry: BaseGeometry, gdf: gpd.GeoDataFrame
    ) -> bool:  # pragma: no cover
        pass

    def fix(self, geometry: BaseGeometry):  # pragma: no cover
        pass

    def skip(self, geometry: BaseGeometry):
        """Checks whether to skip the check. Used for skipping check that only works for certain types."""
        # If nothing is specified always, run validator
        if self.geometry_types is None:
            return False
        elif geometry.geom_type in self.geometry_types:
            return False
        else:
            return True


@patch
def validate(
    self: BaseValidator, gdf: gpd.GeoDataFrame, clone=True
) -> gpd.GeoDataFrame:
    """Method that checks the validity of a each geometry and applies a fix to these geometries or raise a warning"""
    if clone:
        gdf = gdf.copy()
    check_arguments = self.get_check_arguments(gdf)
    is_valid = gdf.geometry.apply(
        lambda g: self.skip(g) or self.check(g, **check_arguments),
    )
    if self.add_new_column:
        gdf[self.validator_column_name] = is_valid

    # For cases where no fix is available, run warning instead of applying fixes
    if (not self.fix_available) and (~is_valid.all()):
        warnings.warn(self.warning_message)
    # Fix geometries
    elif self.apply_fix:
        gdf.loc[~is_valid, "geometry"] = gdf[~is_valid].geometry.apply(self.fix)
    return gdf

In [6]:
# export


class OrientationValidator(BaseValidator):
    """Checks and fixes Orienation of the geometry to ensure it works for multiple system"""

    validator_column_name = "is_oriented_properly"
    geometry_types = ["MultiPolygon", "Polygon"]


@patch
def check(self: OrientationValidator, geometry: BaseGeometry) -> bool:
    """Checks if orientation is counter clockwise"""
    if geometry.geom_type == "Polygon":
        return signed_area(geometry.exterior) >= 0
    elif geometry.geom_type == "MultiPolygon":
        return all([signed_area(g.exterior) >= 0 for g in geometry.geoms])


@patch
def fix(self: OrientationValidator, geometry: BaseGeometry) -> BaseGeometry:
    """Fixes orientation if orientation is clockwise"""
    if geometry.geom_type == "Polygon":
        return orient(geometry)
    elif geometry.geom_type == "MultiPolygon":
        return MultiPolygon([orient(g) for g in geometry.geoms])

In [7]:
# export


class CrsBoundsValidator(BaseValidator):
    """Checks bounds of the geometry to ensure it is within bounds or crs"""

    validator_column_name = "is_within_crs_bounds"
    fix_available = False
    warning_message = "Found geometries out of bounds from crs"

    def get_check_arguments(self, gdf: gpd.GeoDataFrame) -> dict:
        return {"gdf": gdf}


@patch
def check(
    self: CrsBoundsValidator, geometry: BaseGeometry, gdf: gpd.GeoDataFrame
) -> bool:
    """Checks if polygon is within bounds of crs"""
    xmin, ymin, xmax, ymax = gdf.crs.area_of_use.bounds
    b_xmin, b_ymin, b_xmax, b_ymax = geometry.bounds
    return (
        (b_xmin >= xmin) and (b_ymin >= ymin) and (b_xmax <= xmax) and (b_ymax <= ymax)
    )


@patch
def fix(
    self: CrsBoundsValidator, geometry: BaseGeometry
) -> BaseGeometry:  # pragma: no cover
    """No fix available for CRS Bounds"""
    return geometry

In [8]:
# export


class SelfIntersectingValidator(BaseValidator):
    """Checks bounds of the geometry to ensure it is within bounds or crs"""

    validator_column_name = "is_not_self_intersecting"


@patch
def check(self: SelfIntersectingValidator, geometry: BaseGeometry) -> bool:
    explanation = shapely_validation.explain_validity(geometry)
    return "Self-intersection" not in explanation


@patch
def fix(self: SelfIntersectingValidator, geometry: BaseGeometry) -> BaseGeometry:
    """Fix intersection geometry by applying shapely.validation.make_valid"""
    return shapely_validation.make_valid(geometry)

In [9]:
# export


class NullValidator(BaseValidator):
    """Checks bounds of the geometry to ensure it is within bounds or crs"""

    validator_column_name = "is_not_null"
    fix_available = False
    warning_message = "Found null geometries"


@patch
def check(self: NullValidator, geometry: BaseGeometry) -> bool:
    """Checks if polygon is within bounds of crs"""
    return not pd.isnull(geometry)


@patch
def fix(
    self: NullValidator, geometry: BaseGeometry
) -> BaseGeometry:  # pragma: no cover
    """No fix available for CRS Bounds"""
    return geometry

In [10]:
# export


class GeometryValidation:
    """Applies a list of validation checks and tries to fix them"""

    validators_map = {
        "orientation": OrientationValidator,
        "crs_bounds": CrsBoundsValidator,
        "self_intersecting": SelfIntersectingValidator,
        "null": NullValidator,
    }

    def __init__(
        self,
        gdf: gpd.GeoDataFrame,
        validators: Sequence[Union[str, BaseValidator]] = (
            "null",
            "self_intersecting",
            "orientation",
            "crs_bounds",
        ),
        add_validation_columns: bool = True,
        apply_fixes: bool = True,
    ) -> gpd.GeoDataFrame:
        self.gdf = gdf
        self.validators = validators
        self.add_validation_columns = add_validation_columns
        self.apply_fixes = apply_fixes

    def _get_validators(self) -> Sequence[BaseValidator]:
        """Gets a list of Validator Classes based on string"""
        validators_classes = []
        for validator in self.validators:
            if isinstance(validator, str):
                if validator not in self.validators_map:
                    raise ValidationError("Invalid validator.")
                validators_classes.append(self.validators_map[validator])
            elif issubclass(validator, BaseValidator):
                validators_classes.append(validator)
            if issubclass(validator, BaseValidator):
                validators_classes.append(validator)
            else:
                raise ValidationError("Invalid validator.")
        return validators_classes


@patch
def validate_all(self: GeometryValidation) -> gpd.GeoDataFrame:
    """Sequentially run validators"""
    validators = self._get_validators()
    gdf = self.gdf
    for validator in validators:
        gdf = validator(
            add_new_column=self.add_validation_columns,
            apply_fix=self.apply_fixes,
        ).validate(gdf)
    return gdf

In [11]:
# hide
from nbdev.export import notebook2script

notebook2script("00_validation.ipynb")

Converted 00_validation.ipynb.
